_____

In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time
import requests
import threading
import random
import os

# 필요한 라이브러리 설치 안내
try:
    from webdriver_manager.chrome import ChromeDriverManager
except ModuleNotFoundError:
    print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    exit()

In [9]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [13]:

## 기본적으로 '짤'로 필터링한 이미지들로 가지고 오기
## 다음인 '밈'으로 이미지들 가지고오기 
## 5페이지 
## 한번에 50000개 가져오면 됨 


# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "무한도전"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5          # 병렬로 실행할 스레드 개수
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def scroll_and_collect_images(thread_index):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        pinterest_login(driver)
        #driver.get(f"https://www.pinterest.com/search/pins/?q={SEARCH_KEYWORD}")
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD} 짤&rs=guide&journey_depth=1&source_module_id=OB_{SEARCH_KEYWORD}_짤_6b2cd16b-5da6-4ee6-8f3d-f69cf26881af&add_refine={SEARCH_KEYWORD}|guide|word|1")
        #https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD} 짤&rs=guide&journey_depth=1&source_module_id=OB_{SEARCH_KEYWORD}_짤_6b2cd16b-5da6-4ee6-8f3d-f69cf26881af&add_refine={SEARCH_KEYWORD}|guide|word|1

        image_urls = set()
        scroll_pause_time = random.uniform(1, 3)  # 무작위 대기 시간 설정
        retries = 3  # 최대 재시도 횟수
        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                
                # 이미지 수집 (요소 다시 찾기)
                image_elements = driver.find_elements(By.TAG_NAME, "img")
                for img_element in image_elements:
                    img_url = img_element.get_attribute("src")
                    if img_url and img_url.startswith("https") and img_url not in image_urls:
                        image_urls.add(img_url)
                        if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                            break
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(2)
        
        # 이미지 저장
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                urllib.request.urlretrieve(img_url, save_path)
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i,))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Retrying scroll in thread 0 due to error: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=130.0.6723.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001047e7648 cxxbridge1$str$ptr + 3645404
1   chromedriver                        0x00000001047dfea8 cxxbridge1$str$ptr + 3614780
2   chromedriver                        0x000000010424c104 cxxbridge1$string$len + 88416
3   chromedriver                        0x000000010425119c cxxbridge1$string$len + 109048
4   chromedriver                        0x0000000104252b0c cxxbridge1$string$len + 115560
5   chromedriver                        0x00000001042c8808 cxxbridge1$string$len + 598116
6   chromedriver                        0x00000001042c7bd0 cxxbridge1$string$len + 594988
7   chromedriver                        0x0000000

KeyboardInterrupt: 

In [19]:


# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "무한도전"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 3          # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        # 헤드리스 모드 옵션 제거
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")

        image_urls = set()
        scroll_pause_time = random.uniform(1, 3)  # 무작위 대기 시간 설정
        retries = 3  # 최대 재시도 횟수
        wait = WebDriverWait(driver, 10)

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(2)
        
        # 이미지 저장
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    print(f"Thread {thread_index}: Downloaded {progress[thread_index]} images. Total: {total_downloaded}/{NUMBER_OF_IMAGES}")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 0: Downloaded 140 images. Total: 395/10000Failed to download image 139 in thread 0: Socket operation on non-socket
Thread 1: Downloaded 158 images. Total: 396/10000
Thread 0: Downloaded 141 images. Total: 397/10000
Thread 1: Downloaded 159 images. Total: 398/10000
Thread 0: Downloaded 142 images. Total: 399/10000
Thread 1: Downloaded 160 images. Total: 400/10000
Thread 0: Downloaded 143 images. Total: 401/10000
Thread 1: Downloaded 161 images. Total: 402/10000
Thread 0: Downloaded 144 images. Total: 403/10000
Thread 1: Downloaded 162 images. Total: 404/10000
Thread 1: Downloaded 163 images. Total: 405/10000
Thread 0: Downloaded 145 images. Total: 406/10000
Thread 2: Downloaded 99 images. Total: 407/10000
Thread 0: Downloaded 146 images. Total: 408/10000
Thread 1: Downloaded 164 images. Total: 409/10000
Thread 2: Downloaded 100 images. Total: 410/10000
Thread 0: Downloaded 147 images. Total: 411/10000
Thread 1: Downloaded 165 images. Total: 412/10000
Thread 2: Downloaded 101 imag

KeyboardInterrupt: 

In [23]:
# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "무한도전"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 3          # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")

        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    print(f"Thread {thread_index}: Downloaded {progress[thread_index]} images. Total: {total_downloaded}/{NUMBER_OF_IMAGES}")
                    if total_downloaded % 100 == 0:
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
                    if progress[thread_index] % 100 == 0:
                        print(f"Thread {thread_index}: {progress[thread_index]} images downloaded so far.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: Downloaded 1 images. Total: 1/10000
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: Downloaded 1 images. Total: 2/10000
Thread 2: Downloaded 2 images. Total: 3/10000
Thread 1: Downloaded 2 images. Total: 4/10000
Thread 2: Downloaded 3 images. Total: 5/10000
Thread 1: Downloaded 3 images. Total: 6/10000
Thread 2: Downloaded 4 images. Total: 7/10000
Thread 1: Downloaded 4 images. Total: 8/10000
Thread 2: Downloaded 5 images. Total: 9/10000
Thread 1: Downloaded 5 images. Total: 10/10000
Thread 2: Downloaded 6 images. Total: 11/10000
Thread 1: Downloaded 6 images. Total: 12/10000
Thread 2: Downloaded 7 images. Total: 13/10000Thread 1: Downloaded 7 images. Total: 14/10000

Thread 2: Downloaded 8 images. Total: 15/10000
Thread 1: Downloaded 8 images. Total: 16/10000
Thread 1: Downloaded 9 images. Total: 17/10000
Thread 2: Downloaded 9 images. Total: 18/10000
Thread 2: Downloaded 10 images. Total: 19/10000
Thread 1: Downloaded 10 images. Total: 20/100

In [27]:
# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "무한도전"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")

        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/10000 images downloaded.
Progress: 1.00% completed.
Total: 200/10000 images downloaded.
Progress: 2.00% completed.
Total: 300/10000 images downloaded.
Progress: 3.00% completed.
Total: 400/10000 images downloaded.
Progress: 4.00% completed.
Total: 500/10000 images downloaded.
Progress: 5.00% completed.
Total: 600/10000 images downloaded.
Progress: 6.00% completed.
Total: 700/10000 images downloaded.
Progress: 7.00% completed.
Total: 800/10000 images downloaded.
Progress: 8.00% completed.
Total: 900/10000 images downloaded.
Progress: 9.00% completed.
Total: 1000/10000 images downloaded.
Progress: 10.00% completed.
Total: 1100/10000 images downloaded.
Progress: 11.00% completed.
Total: 1200/10000 images downloaded.
Progress: 12.00% completed.
Total: 1300/10000 images downloaded.
Progress: 13.00% completed

In [31]:

#루피/ 밈

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "루피"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")

        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.

Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/10000 images downloaded.
Progress: 1.00% completed.
Total: 200/10000 images downloaded.
Progress: 2.00% completed.
Total: 300/10000 images downloaded.
Progress: 3.00% completed.
Total: 400/10000 images downloaded.
Progress: 4.00% completed.
Total: 500/10000 images downloaded.
Progress: 5.00% completed.
Total: 600/10000 images downloaded.
Progress: 6.00% completed.
Total: 700/10000 images downloaded.
Progress: 7.00% completed.
Total: 800/10000 images downloaded.
Progress: 8.00% completed.
Total: 900/10000 images downloaded.
Progress: 9.00% completed.
Total: 1000/10000 images downloaded.
Progress: 10.00% completed.
Total: 1100/10000 images downloaded.
Progress: 11.00% completed.
Total: 1200/10000 images downloaded.
Progress: 12.00% completed.


In [35]:

#무기력 / 짤

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "무기력"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ########################## 여기 수정하면 됨 #####################################
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")

        image_urls = set()
        scroll_pause_time = 2  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 100  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.

Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.


In [37]:

#무기력 / 밈

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "무기력"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ########################## 여기 수정하면 됨 #####################################
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")

        image_urls = set()
        scroll_pause_time = 3  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 100  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.

Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.


In [39]:

#어쩔티비 / 짤
# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "어쩔티비"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ########################## 여기 수정하면 됨 #####################################
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")

        image_urls = set()
        scroll_pause_time = 3  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 100  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/10000 images downloaded.
Progress: 1.00% completed.
Total: 200/10000 images downloaded.
Progress: 2.00% completed.
Total: 300/10000 images downloaded.
Progress: 3.00% completed.


In [45]:
# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "어쩔티비"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")

        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.



In [13]:
## 어쩔티비 

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "어쩔티비"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ####################################################여기 수정 #########################################################3
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}&rs=typed")


        
        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/10000 images downloaded.
Progress: 1.00% completed.
Total: 200/10000 images downloaded.
Progress: 2.00% completed.
Total: 300/10000 images downloaded.
Progress: 3.00% completed.
Total: 400/10000 images downloaded.
Progress: 4.00% completed.
Total: 500/10000 images downloaded.
Progress: 5.00% completed.
Total: 600/10000 images downloaded.
Progress: 6.00% completed.
Total: 700/10000 images downloaded.
Progress: 7.00% completed.
Total: 800/10000 images downloaded.
Progress: 8.00% completed.
Total: 900/10000 images downloaded.
Progress: 9.00% completed.
Total: 1000/10000 images downloaded.
Progress: 10.00% completed.
Total: 1100/10000 images downloaded.
Progress: 11.00% completed.
Total: 1200/10000 images downloaded.
Progress: 12.00% completed.
Total: 1300/10000 images downloaded.
Progress: 13.00% completed

In [15]:
## 어쩔티비 
## 짤_a

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "어쩔티비"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images_a"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ####################################################여기 수정 #########################################################3
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}&rs=typed")


        
        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}_a.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/10000 images downloaded.
Progress: 1.00% completed.
Total: 200/10000 images downloaded.
Progress: 2.00% completed.
Total: 300/10000 images downloaded.
Progress: 3.00% completed.
Total: 400/10000 images downloaded.
Progress: 4.00% completed.
Total: 500/10000 images downloaded.
Progress: 5.00% completed.
Total: 600/10000 images downloaded.
Progress: 6.00% completed.
Total: 700/10000 images downloaded.
Progress: 7.00% completed.
Total: 800/10000 images downloaded.
Progress: 8.00% completed.
Total: 900/10000 images downloaded.
Progress: 9.00% completed.
Total: 1000/10000 images downloaded.
Progress: 10.00% completed.
Total: 1100/10000 images downloaded.
Progress: 11.00% completed.
Total: 1200/10000 images downloaded.
Progress: 12.00% completed.
Total: 1300/10000 images downloaded.
Progress: 13.00% completed

In [17]:
## 어쩔티비 
## 밈_b

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "어쩔티비"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images_b"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ####################################################여기 수정 #########################################################3
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}&rs=typed")


        
        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}_b.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")


Thread 2: 스크롤 한계에 도달하여 크롤링을 종료합니다.Thread 0: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 4: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Thread 3: 스크롤 한계에 도달하여 크롤링을 종료합니다.

Thread 1: 스크롤 한계에 도달하여 크롤링을 종료합니다.
Total: 100/10000 images downloaded.
Progress: 1.00% completed.
Total: 200/10000 images downloaded.
Progress: 2.00% completed.
Total: 300/10000 images downloaded.
Progress: 3.00% completed.
Total: 400/10000 images downloaded.
Progress: 4.00% completed.
Total: 500/10000 images downloaded.
Progress: 5.00% completed.
Total: 600/10000 images downloaded.
Progress: 6.00% completed.
Total: 700/10000 images downloaded.
Progress: 7.00% completed.
Total: 800/10000 images downloaded.
Progress: 8.00% completed.
Total: 900/10000 images downloaded.
Progress: 9.00% completed.
Total: 1000/10000 images downloaded.
Progress: 10.00% completed.
Total: 1100/10000 images downloaded.
Progress: 11.00% completed.
Total: 1200/10000 images downloaded.
Progress: 12.00% completed.
Total: 1300/10000 images downloaded.
Progress: 13.00% completed

In [ ]:
## 어쩔티비 
## 추가 검색어 없음_c

# Pinterest 로그인 정보 설정
PINTEREST_EMAIL = "mountainstar03@hufs.ac.kr"
PINTEREST_PASSWORD = "byeol617!"

# 크롤링 설정
SEARCH_KEYWORD = "어쩔티비"  # 검색어 설정
NUMBER_OF_IMAGES = 10000  # 크롤링할 이미지 개수 설정
THREAD_COUNT = 5        # 병렬로 실행할 스레드 개수 (줄여서 안정성 증가)
SAVE_DIR = "pinterest_images_c"  # 이미지 저장 폴더 경로

# 크롤링할 저장 경로 생성
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def pinterest_login(driver):
    driver.get("https://www.pinterest.com/login/")
    time.sleep(2)
    
    email_input = driver.find_element(By.NAME, "id")
    password_input = driver.find_element(By.NAME, "password")
    
    email_input.send_keys(PINTEREST_EMAIL)
    password_input.send_keys(PINTEREST_PASSWORD)
    
    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()
    
    time.sleep(5)  # 로그인 후 대기

def download_image(url, save_path, retries=3):
    while retries > 0:
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                return True
            else:
                retries -= 1
        except Exception as e:
            retries -= 1
            print(f"Retry downloading {url}, retries left: {retries}")
    return False

def scroll_and_collect_images(thread_index, progress):
    driver = None
    try:
        from webdriver_manager.chrome import ChromeDriverManager
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        pinterest_login(driver)
        ####################################################여기 수정 #########################################################3
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20밈")
        #driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}%20짤")
        driver.get(f"https://kr.pinterest.com/search/pins/?q={SEARCH_KEYWORD}&rs=typed")


        
        image_urls = set()
        scroll_pause_time = 1  # 스크롤 대기 시간을 최소화하여 속도 증가
        retries = 5  # 최대 재시도 횟수 증가
        wait = WebDriverWait(driver, 10)
        scroll_attempts = 0
        max_scroll_attempts = 50  # 최대 스크롤 시도 횟수 증가로 더 많은 이미지 확보

        while len(image_urls) < NUMBER_OF_IMAGES // THREAD_COUNT:
            try:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
                time.sleep(scroll_pause_time)
                scroll_attempts += 1
                
                # 이미지 수집 (명시적 대기 사용)
                image_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
                for img_element in image_elements:
                    try:
                        img_url = img_element.get_attribute("src")
                        if img_url and img_url.startswith("https") and img_url not in image_urls:
                            image_urls.add(img_url)
                            if len(image_urls) >= NUMBER_OF_IMAGES // THREAD_COUNT:
                                break
                    except StaleElementReferenceException:
                        # 요소가 유효하지 않으면 무시하고 계속 시도
                        continue

                # 최대 스크롤 시도에 도달한 경우 종료
                if scroll_attempts >= max_scroll_attempts:
                    print(f"Thread {thread_index}: 스크롤 한계에 도달하여 크롤링을 종료합니다.")
                    break
            except StaleElementReferenceException:
                # 스크롤 중 요소가 유효하지 않으면 무시하고 다시 시도
                continue
            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to scroll in thread {thread_index} after multiple retries: {e}")
                    break
                else:
                    print(f"Retrying scroll in thread {thread_index} due to error: {e}")
                    time.sleep(1)
        
        # 이미지 저장 (수정된 부분)
        for idx, img_url in enumerate(image_urls):
            try:
                save_path = os.path.join(SAVE_DIR, f"{thread_index}_image_{idx}_c.jpg")
                if download_image(img_url, save_path):
                    progress[thread_index] += 1
                    total_downloaded = sum(progress)
                    # 100개 단위로만 출력
                    if total_downloaded % 100 == 0:
                        print(f"Total: {total_downloaded}/{NUMBER_OF_IMAGES} images downloaded.")
                        print(f"Progress: {total_downloaded / NUMBER_OF_IMAGES * 100:.2f}% completed.")
            except Exception as e:
                print(f"Failed to download image {idx} in thread {thread_index}: {e}")
    except ModuleNotFoundError as e:
        print(f"Module 'webdriver_manager' not found in thread {thread_index}: {e}")
    except Exception as e:
        print(f"An error occurred in thread {thread_index}: {e}")
    finally:
        if driver is not None:
            driver.quit()

def start_crawling():
    threads = []
    progress = [0] * THREAD_COUNT  # 각 스레드의 진행 상황을 추적
    for i in range(THREAD_COUNT):
        thread = threading.Thread(target=scroll_and_collect_images, args=(i, progress))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    try:
        start_crawling()
    except ModuleNotFoundError as e:
        print("Module 'webdriver_manager' not found. Please install it using 'pip install webdriver-manager'.")
    except Exception as e:
        print(f"An error occurred: {e}")
